In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

load data

In [26]:
df = pd.read_csv('Dataset .csv')

preprocessing data

In [27]:
df.fillna({
    'Cuisines': 'Unknown',
    'City': 'Unknown',
    'Average Cost for two': df['Average Cost for two'].median(),
    'Rating text': 'Not rated',
    'Has Table booking': 'No'
}, inplace=True)

encode categorical variables

In [28]:
label_encoders = {}
for col in ['Cuisines', 'City']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

df_display = df.copy()
df_display['Cuisines'] = label_encoders['Cuisines'].inverse_transform(df['Cuisines'])
df_display['City'] = label_encoders['City'].inverse_transform(df['City'])

user input

In [29]:
print("\nAvailable Cuisines:", list(label_encoders['Cuisines'].classes_))
print("Available Cities:", list(label_encoders['City'].classes_))
print("Available Ratings: ['Excellent', 'Very Good', 'Good', 'Average', 'Poor', 'Not rated']")

user_cuisine = input("\nEnter preferred cuisine (e.g., Japanese): ").strip().lower()
user_city = input("Enter preferred city (e.g., Makati City): ").strip().lower()
user_budget = float(input("Enter your budget for two (e.g., 1500): "))
user_rating_filter = input("Minimum acceptable rating (e.g., Excellent): ").strip().lower()
table_booking = input("Require table booking? (Yes/No): ").strip().lower()


Available Cuisines: ['Afghani', 'Afghani, Mughlai, Chinese', 'Afghani, North Indian', 'Afghani, North Indian, Pakistani, Arabian', 'African', 'African, Portuguese', 'American', 'American, Asian, Burger', 'American, Asian, European, Seafood', 'American, Asian, Italian, Seafood', 'American, Asian, North Indian, Mexican, South Indian', 'American, BBQ', 'American, BBQ, Burger', 'American, BBQ, Sandwich', 'American, BBQ, Seafood', 'American, BBQ, Southern', 'American, BBQ, Steak', 'American, Bakery, European, Burger, Fusion', 'American, Bar Food', 'American, Bar Food, BBQ', 'American, Bar Food, Sandwich', 'American, Breakfast', 'American, Breakfast, Burger', 'American, Breakfast, Desserts', 'American, Breakfast, Diner', 'American, Breakfast, Greek', 'American, Breakfast, Hawaiian', 'American, Breakfast, Vegetarian', 'American, British, Continental', 'American, Burger', 'American, Burger, Cafe', 'American, Burger, Cajun', 'American, Burger, Fast Food', 'American, Burger, Grill', 'American, 

recommend restaurant

In [30]:
def get_closest_match(user_input, valid_options):
    for option in valid_options:
        if option.lower() == user_input:
            return option
    raise ValueError(f"'{user_input}' not found. Valid options are: {valid_options}")

try:
    cuisine_input = get_closest_match(user_cuisine, label_encoders['Cuisines'].classes_)
    city_input = get_closest_match(user_city, label_encoders['City'].classes_)

    encoded_cuisine = label_encoders['Cuisines'].transform([cuisine_input])[0]
    encoded_city = label_encoders['City'].transform([city_input])[0]

   
    user_input_vector = np.array([[encoded_cuisine, encoded_city, user_budget]])
    features = df[['Cuisines', 'City', 'Average Cost for two']]

    similarity_scores = cosine_similarity(user_input_vector, features)
    df['similarity'] = similarity_scores[0]

   
    filtered_df = df[
        (df['Rating text'].str.lower() == user_rating_filter) &
        (df['Has Table booking'].str.lower() == table_booking)
    ]

    if filtered_df.empty:
        print("\n⚠️ No restaurants match all filters. Showing similar ones instead...\n")
        filtered_df = df.copy()

    top_recommendations = filtered_df.sort_values(by='similarity', ascending=False).head(5)

    print("\n🎯 Top Restaurant Recommendations for You:\n")
    print(df_display.loc[top_recommendations.index, [
        'Restaurant Name', 'Cuisines', 'City',
        'Average Cost for two', 'Aggregate rating', 'Rating text', 'Has Table booking'
    ]])

except Exception as e:
    print("\n❌ Error:", e)


🎯 Top Restaurant Recommendations for You:

        Restaurant Name                     Cuisines              City  \
2099  Indian Grill Room        North Indian, Mughlai           Gurgaon   
1839          Prankster  Modern Indian, North Indian           Gurgaon   
0      Le Petit Souffle   French, Japanese, Desserts       Makati City   
4           Sambo Kojin             Japanese, Korean  Mandaluyong City   
9154  The Urban Foundry          North Indian, Asian              Pune   

      Average Cost for two  Aggregate rating Rating text Has Table booking  
2099                  1800               4.5   Excellent               Yes  
1839                  1500               4.8   Excellent               Yes  
0                     1100               4.8   Excellent               Yes  
4                     1500               4.8   Excellent               Yes  
9154                  1500               4.5   Excellent               Yes  
